In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
#         print(x.shape)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) # 1, 1, 32, 32 -> 1, 6, 15, 15
#         print(x.shape)
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # 1, 6, 15, 15 -> 1, 16, 6, 6
#         print(x.shape)
#         print('self.num_flat_features:', self.num_flat_features(x))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [86]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [87]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0541,  0.0811,  0.0482, -0.1162,  0.1373, -0.0362,  0.0467,  0.0928,
          0.0063, -0.1188]], grad_fn=<AddmmBackward>)


In [88]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [89]:
print(out)

tensor([[-0.0541,  0.0811,  0.0482, -0.1162,  0.1373, -0.0362,  0.0467,  0.0928,
          0.0063, -0.1188]], grad_fn=<AddmmBackward>)


In [91]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
print(target.shape)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

torch.Size([1, 10])
tensor(1.1134, grad_fn=<MseLossBackward>)


In [92]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [93]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0101,  0.0054,  0.0047, -0.0019,  0.0077,  0.0022])


In [94]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [95]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

**矩阵大小变化过程**

In [ ]:
# # 定义 Convolution Network 模型
# class Cnn(nn.Module):
#     def __init__(self, in_dim, n_class):
#         super(Cnn, self).__init__()
#         self.conv = nn.Sequential(
#             nn.Conv2d(
#                 in_channels=in_dim,
#                 out_channels=6,
#                 kernel_size=3,
#                 stride=1,
#                 padding=1,
#                 dilation=1,
#                 groups=1,
#                 bias=True,
#                 padding_mode='zeros',), # batch,1, 6, 28, 28
#             nn.ReLU(True),
#             nn.MaxPool2d(2, 2), # batch, 1, 6, 14, 14
#             nn.Conv2d(6, 16, 5, stride=1, padding=0), #batch, 1, 16, 10, 10, 没有padding, 卷积核是5x5, 所以每个边减去2，卷积结束后14x14-> 10 x10
#             nn.ReLU(True),
#             nn.MaxPool2d(2, 2)) #batch, 16, 5, 5
        
#         self.fc = nn.Sequential(
#             nn.Linear(400, 120), # ???为啥是400
#             nn.Linear(120, 84),
#             nn.Linear(84, n_class))
        
#     def forward(self, x):
#         print(x.shape)
#         out = self.conv(x) # 1, 1, 28, 28 -> [1, 16, 5, 5]
#         print(out.shape) 
#         out = out.view(out.size(0), -1) # [1, 400]
#         print(out.shape)
#         out = self.fc(out)
#         return out
# cnn = Cnn(1, 10)

# input = torch.randn(1, 1, 28, 28)
# out = cnn(input)
# print(out)